In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame
import re
from datetime import datetime
import time
import psycopg2
from typing import List, Union

import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")

from sqlalchemy import create_engine

In [ ]:
# a list of required cities

list_city = ['Москва','Санкт-Петербург','Омск','Тимашёвск', 'Хельсинки', 'Икаалинен',  'Учкудук', 'Янгиабад']

In [ ]:
# a list of required values of city (columns)
list_headers = ['Город', 'Страна', 'Основан', 'Тип климата', 'Часовой пояс', 
                'Население','Телефонный код', 'Почтовые индексы']


In [ ]:
#scraing wiki common data for each city 
def extract_data(city: int) -> List[str]:
    wiki = 'https://ru.wikipedia.org/wiki/{}'
    website_url = requests.get(wiki.format(list_city[city])).text
    soup = BeautifulSoup(website_url,'lxml')
    table = soup.find('table',{'class':'infobox'})

    headers = []

    data_headers = table.find_all('tr')

    list_headers_fix = ['Город', 'Страна', 'Основан', 'Тип климата', 'Часовой пояс', 
                'Население','Телефонный код', 'Почтовые индексы', 'Почтовый индекс']
    for header in data_headers:
        value_1 = header.find_all('th')
        value_2 = header.find_all('td')
        beautified_value_1 = [ele.text.strip() for ele in value_1]
        beautified_value_2 = [ele.text.strip() for ele in value_2]
        # Remove data arrays that are empty
        if len(beautified_value_1) == 0 or len(beautified_value_2) == 0:
            continue    
        headers.append(beautified_value_1)
        headers.append(beautified_value_2)

                
        headers_new = [elem for row in headers for elem in row]        

        final_list = []
        for elem in list_headers_fix:
             if elem in headers_new:
                    k = headers_new.index(elem)
                    final_list.append(elem)
                    final_list.append(headers_new[k+1])
        final_list.insert(0, list_city[city%len(list_city)])
    return final_list

In [ ]:
#filter required columns
def city_info(final_list: List[str]) -> List[str]:

    list = {0:'',1:'',2:'',3:'',4:'',5:'',6:'',7:''}
    
    for i, value in enumerate(final_list):
        if value == 'Почтовый индекс':
            final_list[i] = 'Почтовые индексы'

    list[0] = final_list[0]
    for i in range(1, len(final_list)):
        if final_list[i] in list_headers: list[list_headers.index(final_list[i])] = final_list[i+1]
    result = [list[key] for key in list]
           
    return result
   
    

In [ ]:
# function for making format of city index 
def get_city_index(row: str) -> Union[str, None]:
    if re.search('х', row): result = row.replace("х", "0")+"—"+row.replace("х", "9") 
    elif re.search('x', row): result = row.replace("x", "0")+"—"+row.replace("x", "9")  
    else: result = row.replace("[7]", "")
    return(result)

In [ ]:
#create a DataFrame

city_df = pd.DataFrame(columns=list_headers)

for i in range(len(list_city)): 
    data = city_info(extract_data(i))
    city_df.loc[len(city_df.index)] = data 
    

In [ ]:
def get_format_table(city_df: pd.DataFrame) -> pd.DataFrame:
    
        #convert data to normal format

        city_df['Основан'] = (city_df['Основан'].apply(lambda x: re.sub("[^0-9]","", x)[-4:]))
        city_df['Часовой пояс'] = (city_df['Часовой пояс'].apply(lambda x: re.sub("[^0-9]","", x)[-3:-2]))
        city_df['Население'] = (city_df['Население'].apply(lambda x: re.sub("[^0-9]","", x)[:-5]))
        city_df['Почтовые индексы'] = (city_df['Почтовые индексы'].apply(lambda x: get_city_index(x)))


        #adding date and timestamp columns

        city_df = city_df.assign(Date = datetime.now().date())
        city_df = city_df.assign(Timestamp = time.time()) #pd.Timestamp(a, unit='s')


        #columns naming

        city_df = city_df.rename(columns={'Город': 'city',
                                        'Страна': 'Country',
                                        'Основан': 'Founded',
                                        'Тип климата': 'Climate',
                                        'Часовой пояс': 'UTC',
                                        'Население': 'Population',
                                        'Телефонный код': 'Phonecode',
                                        'Почтовые индексы': 'Postcode'
                                       })


        # type changing
        try:
            #city_df.Founded = city_df.Founded.astype('int') 
            city_df.UTC = city_df.UTC.astype('int') 
            city_df.Population = city_df.Population.astype('int')
            city_df.Date = pd.to_datetime(city_df.Date)
        except Exception as error:
            print(error)
            
        return city_df

In [ ]:
get_format_table(city_df)

#### Connecting to PostgreSQL Database using SQLAlchemy in Python

In [ ]:
# DEFINE THE DATABASE CREDENTIALS
user = 'postgres'
password = '123456'
host = 'localhost'
port = 5432
database = 'db_test'

In [ ]:
# CONNECTION TO DB
try:
    con = psycopg2.connect (
        host = host,
        dbname = database,
        user = user,
        password = password,
        port = port)
    #con.close()
except Exception as error:
    print(error)
    
cur = con.cursor()
# cur = con.cursor(cursor_factory = psycopg2.extras.DictCursor
# con.commit()

In [ ]:
# PYTHON FUNCTION TO CONNECT TO THE POSTGRESQL DATABASE AND
# RETURN THE SQLACHEMY ENGINE OBJECT
def get_connection():
    return create_engine(
        url="postgresql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
    )
 

In [ ]:
if __name__ == '__main__':
 
    try:
        # GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
        engine = get_connection()
        print(
            f"Connection to the {host} for user {user} created successfully.")
    except Exception as ex:
        print("Connection could not be made due to the following error: \n", ex)

In [ ]:
# download postgresql table
data = city_df
data.to_sql('wiki_city', engine, if_exists='replace')

#### Data from Postgres and display Python

In [ ]:
query = 'SELECT * FROM wiki_city'
df_postgres = pd.read_sql(query, con = con)


In [ ]:
df_postgres.drop(columns=['index'], inplace=True)

In [ ]:
df_postgres